In [26]:
import pandas as pd
import re
list_csv = ['data/kor/attraction_places.csv',
            'data/kor/hotel_places.csv',
            'data/kor/restaurant_places.csv',
            'data/eng/eng_attraction_places.csv',
            'data/eng/eng_hotel_places.csv',
            'data/eng/eng_restaurant_places.csv']
review_csv = ['data/kor/attraction_review.csv',
              'data/kor/hotel_review.csv',
              'data/kor/restaurant_review.csv',
              'data/eng/eng_attraction_review.csv',
              'data/eng/eng_hotel_review.csv',
              'data/eng/eng_restaurant_review.csv']
save_csv = ['data/attraction_places.csv',
            'data/hotel_places.csv',
            'data/restaurant_places.csv']
save2_csv = ['data/attraction_dropped.csv',
             'data/hotel_dropped.csv',
             'data/restaurant_dropped.csv']

In [49]:
for i in range(3):
    df_kor = pd.read_csv(list_csv[i],
                         names=['placeId', 'name_kor', 'location', 'class'],
                         encoding='euc-kr')
    tmp_class = df_kor['class'][0]
    df_eng = pd.read_csv(list_csv[i+3],
                         names=['placeId', 'name_eng', 'location', 'class'],
                         encoding='euc-kr')

    df_eng = df_eng.iloc[:, [j for j, c in enumerate(df_eng.columns) if j != 4]]
    df_eng = df_eng.drop(columns=['location', 'class'])
    # df_kor, df_eng 병합
    df = pd.merge(df_kor, df_eng, how='outer')
    # column 순서 재배치
    cols = df.columns.tolist()
    cols = cols[3:4] + cols[0:2] + cols[-1:] + cols[2:3]
    df= df[cols]
    df['class'] = df['class'].fillna(tmp_class)
    # placeId string으로 변환
    #df['placeId'] = df.placeId.astype('str')
    #점수column 생성
    df_kor = pd.read_csv(review_csv[i],
                                names=['placeId', 'userId', 'score', 'title', 'review'],
                                encoding='euc-kr')
    df_kor= df_kor.drop(columns=['userId', 'title', 'review'])
    df_kor['score'] = (df_kor['score']/10).astype('int64')
    df_eng = pd.read_csv(review_csv[i+3],
                                names=['placeId', 'userId', 'score', 'title', 'review'],
                                encoding='euc-kr')
    df_eng = df_eng.drop(columns=['userId', 'title', 'review'])
    df_eng['score'] = (df_eng['score']/10).astype('int64')
    df_kor = df_kor.append(df_eng)
    df_score = df_kor.groupby(['placeId'],as_index=False).mean()
    #df_score['placeId'] = df_score['placeId'].astype('str')
    
    #점수 column 병합
    df = df.merge(df_score, how='inner')
    cols = df.columns.tolist()
    df = df[cols[0:2]+cols[5:6]+cols[2:5]]

    # 테스트용: 좌표 외 drop
    df_dropped = df.dropna(subset=['location'])
    series_match = df_dropped['location'].apply(lambda x:
                                                re.fullmatch('[0-9]+\.[0-9]+,[0-9]+\.[0-9]+', x))
    df_dropped = df_dropped[pd.notnull(series_match)]

    # 좌표 (float,float)형 리스트로 변환
    df_location = pd.DataFrame(df_dropped['location'].apply(lambda x:
                                                            re.findall('[0-9]+\.[0-9]+', x)))
    df_dropped['latitude'] = df_location['location'].apply(lambda x: x[0])
    df_dropped['longitude'] = df_location['location'].apply(lambda x: x[1])
    df_dropped = df_dropped.drop(columns=['location'])
    ###
    df.to_csv(save_csv[i], index=False)
    df_dropped.to_csv(save2_csv[i], index=False)
    print('before drop: ', len(df))
    print('after  drop: ', len(df_dropped))
    print()

before drop:  568
after  drop:  351

before drop:  1043
after  drop:  607

before drop:  23162
after  drop:  18245



In [50]:
type(df['placeId'][0])

numpy.int64

In [16]:
# print('전체',len(df_kor))
print('NA',len(df[pd.isna(df['location'])]))
df_tmp = df.dropna(subset=['location'])
series_match = df_tmp['location'].apply(lambda x:
                                        re.fullmatch('[0-9]+\.[0-9]+,[0-9]+\.[0-9]+', x))
print('좌표',len(df_tmp[pd.notnull(series_match)]))
print('주소',len(df_tmp[pd.isnull(series_match)]))

NA 22
좌표 18245
주소 4895
